In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Crawl stock price

## Selenium

In [2]:
apple_url = 'https://www.investing.com/equities/apple-computer-inc-historical-data'

In [3]:
browser = webdriver.Chrome(executable_path='chromedriver.exe')
browser.maximize_window()

In [17]:
browser = webdriver.Firefox(executable_path='geckodriver.exe')

In [18]:
browser.get(apple_url)

### Close popup

In [ ]:
browser.find_element_by_class_name('popupCloseIcon.largeBannerCloser').click()

### Click the calendar

In [19]:
browser.find_element_by_id('widgetFieldDateRange').click()

### Find and input start date

In [21]:
startDate = browser.find_element_by_id('startDate')
startDate.clear()
startDate.send_keys('02/15/2001')
browser.find_element_by_id('applyBtn').send_keys(Keys.ENTER)

### Look up value

In [22]:
str_xpath = '/html/body/div[5]/section/div[9]/table[1]/tbody/tr[{}]/td[{}]'

In [23]:
date = []
close = []
Open = []
high = []
low = []

In [ ]:
%%time
for i in range(1, 5000+1):
    for j in range(1, 5+1):
        xpath = str_xpath.format(i, j)
        real_value = browser.find_element_by_xpath(xpath)
        if j == 1:
            date.append(real_value.text)
        if j == 2:
            close.append(real_value.text)
        if j == 3:
            Open.append(real_value.text)
        if j == 4:
            high.append(real_value.text)
        if j == 5:
            low.append(real_value.text)

In [25]:
len(date)

5000

In [26]:
len(close)

5000

In [27]:
len(Open)

5000

In [28]:
len(high)

5000

In [29]:
len(low)

5000

## Save dataframe

In [2]:
import pandas as pd

In [31]:
columns = ['Date', 'Close', 'Open', 'High', 'Low']
aapl_stock_price = pd.DataFrame({
    'Date': date, 
    'Close': close, 
    'Open': Open, 
    'High': high, 
    'Low': low
    })
aapl_stock_price

,Date,Close,Open,High,Low
0,"Dec 29, 2020",136.53,138.05,138.78,135.53
1,"Dec 28, 2020",136.69,133.99,137.34,133.52
2,"Dec 24, 2020",131.97,131.32,133.46,131.10
3,"Dec 23, 2020",130.96,132.18,132.32,130.83
4,"Dec 22, 2020",131.88,131.68,134.40,129.66
...,...,...,...,...,...
4995,"Feb 22, 2001",0.34,0.34,0.35,0.32
4996,"Feb 21, 2001",0.34,0.33,0.36,0.33
4997,"Feb 20, 2001",0.33,0.34,0.35,0.32
4998,"Feb 16, 2001",0.34,0.34,0.35,0.33


In [33]:
aapl_stock_price.to_csv('aapl_stock_price.csv', index=False)

In [3]:
aapl_df = pd.read_csv('aapl_stock_price.csv', index_col=False)

In [6]:
aapl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Date     5000 non-null object
Close    5000 non-null float64
Open     5000 non-null float64
High     5000 non-null float64
Low      5000 non-null float64
dtypes: float64(4), object(1)
memory usage: 195.4+ KB


In [41]:
aapl_df.describe()

,Close,Open,High,Low
count,5000.000000,5000.000000,5000.000000,5000.000000
mean,19.715890,19.708146,19.917360,19.497556
std,24.018699,24.002924,24.293137,23.716058
min,0.230000,0.230000,0.240000,0.230000
25%,2.260000,2.277500,2.307500,2.240000
50%,11.830000,11.795000,11.920000,11.670000
75%,28.362500,28.352500,28.590000,28.082500
max,136.690000,138.050000,138.780000,135.530000


# Stock Indicators

## Trend Indicators 

### SMA
Simple moving average  
http://www.fmlabs.com/reference/default.htm?url=SimpleMA.htm  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html

In [4]:
def SMA(df, time_period):
    indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=time_period)
    sma = df['Close'].rolling(window=indexer, min_periods=time_period).mean()
    df['SMA_' + str(time_period)] = sma
    return df

In [5]:
aapl_df = SMA(aapl_df, 20)
aapl_df

,Date,Close,Open,High,Low,SMA_20
0,"Dec 29, 2020",136.53,138.05,138.78,135.53,126.7815
1,"Dec 28, 2020",136.69,133.99,137.34,133.52,125.9075
2,"Dec 24, 2020",131.97,131.32,133.46,131.10,124.9025
3,"Dec 23, 2020",130.96,132.18,132.32,130.83,124.1055
4,"Dec 22, 2020",131.88,131.68,134.40,129.66,123.3160
...,...,...,...,...,...,...
4995,"Feb 22, 2001",0.34,0.34,0.35,0.32,NaN
4996,"Feb 21, 2001",0.34,0.33,0.36,0.33,NaN
4997,"Feb 20, 2001",0.33,0.34,0.35,0.32,NaN
4998,"Feb 16, 2001",0.34,0.34,0.35,0.33,NaN


### EMA
Exponential moving average  
https://www.fmlabs.com/reference/default.htm?url=ExpMA.htm  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html

In [6]:
def exponential_moving_average(df, time_period):
    EMA = df['Close'].iloc[::-1].ewm(span=time_period, min_periods=time_period, adjust=False).mean()
    df['EMA_' + str(time_period)] = EMA.iloc[::-1]
    return df

In [7]:
aapl_df = exponential_moving_average(aapl_df, 20)
aapl_df

,Date,Close,Open,High,Low,SMA_20,EMA_20
0,"Dec 29, 2020",136.53,138.05,138.78,135.53,126.7815,127.573094
1,"Dec 28, 2020",136.69,133.99,137.34,133.52,125.9075,126.630262
2,"Dec 24, 2020",131.97,131.32,133.46,131.10,124.9025,125.571342
3,"Dec 23, 2020",130.96,132.18,132.32,130.83,124.1055,124.897799
4,"Dec 22, 2020",131.88,131.68,134.40,129.66,123.3160,124.259672
...,...,...,...,...,...,...,...
4995,"Feb 22, 2001",0.34,0.34,0.35,0.32,NaN,NaN
4996,"Feb 21, 2001",0.34,0.33,0.36,0.33,NaN,NaN
4997,"Feb 20, 2001",0.33,0.34,0.35,0.32,NaN,NaN
4998,"Feb 16, 2001",0.34,0.34,0.35,0.33,NaN,NaN


## Mean Reversion Indicators

### Bollinger Bands
https://www.fmlabs.com/reference/default.htm?url=Bollinger.htm  
https://quant.stackexchange.com/a/31905   
https://github.com/Crypto-toolbox/pandas-technical-indicators/blob/master/technical_indicators.py#L95

In [9]:
def bollinger_bands(df, time_period=20, n_std=2):
    indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=time_period)
    rolling_mean = df['Close'].rolling(window=indexer, min_periods=time_period).mean()
    rolling_std  = df['Close'].rolling(window=indexer, min_periods=time_period).std()
    upper_band = rolling_mean + (rolling_std*n_std)
    lower_band = rolling_mean - (rolling_std*n_std)
    df['BB_upper_' + str(time_period)] = upper_band
    df['BB_lower_' + str(time_period)] = lower_band
    return df

In [10]:
aapl_df = bollinger_bands(aapl_df, 20, 2)
aapl_df

,Date,Close,Open,High,Low,SMA_20,EMA_20,BB_upper_20,BB_lower_20
0,"Dec 29, 2020",136.53,138.05,138.78,135.53,126.7815,127.573094,136.337708,117.225292
1,"Dec 28, 2020",136.69,133.99,137.34,133.52,125.9075,126.630262,134.889829,116.925171
2,"Dec 24, 2020",131.97,131.32,133.46,131.10,124.9025,125.571342,133.282848,116.522152
3,"Dec 23, 2020",130.96,132.18,132.32,130.83,124.1055,124.897799,132.685294,115.525706
4,"Dec 22, 2020",131.88,131.68,134.40,129.66,123.3160,124.259672,132.142442,114.489558
...,...,...,...,...,...,...,...,...,...
4995,"Feb 22, 2001",0.34,0.34,0.35,0.32,NaN,NaN,NaN,NaN
4996,"Feb 21, 2001",0.34,0.33,0.36,0.33,NaN,NaN,NaN,NaN
4997,"Feb 20, 2001",0.33,0.34,0.35,0.32,NaN,NaN,NaN,NaN
4998,"Feb 16, 2001",0.34,0.34,0.35,0.33,NaN,NaN,NaN,NaN


## Volume Indicators

### OBV
https://www.fmlabs.com/reference/default.htm?url=OBV.htm  
https://stackoverflow.com/a/60093990/9480586  
https://github.com/Crypto-toolbox/pandas-technical-indicators/blob/master/technical_indicators.py#L417  
OBV need the "Volume" information

## Momentum Indicators

### MACD
Moving Average Convergence Divergence  
https://www.fmlabs.com/reference/default.htm?url=MACD.htm  
https://github.com/Crypto-toolbox/pandas-technical-indicators/blob/master/technical_indicators.py#L219

In [13]:
def macd(df, n_fast, n_slow):
    """Calculate MACD, MACD Signal and MACD difference
    :param df: pandas.DataFrame
    :param n_fast: 
    :param n_slow: 
    :return: pandas.DataFrame
    """
    EMAfast = df['Close'].iloc[::-1].ewm(span=n_fast, min_periods=n_fast, adjust=False).mean().iloc[::-1]
    EMAslow = df['Close'].iloc[::-1].ewm(span=n_slow, min_periods=n_slow, adjust=False).mean().iloc[::-1]
    MACD = EMAfast - EMAslow
    MACDsign = MACD.iloc[::-1].ewm(span=9, min_periods=9, adjust=False).mean().iloc[::-1]
    MACDdiff = MACD - MACDsign
    df['MACD_' + str(n_fast) + '_' + str(n_slow)] = MACD
    df['MACDsign_' + str(n_fast) + '_' + str(n_slow)] = MACDsign
    df['MACDdiff_' + str(n_fast) + '_' + str(n_slow)] = MACDdiff
    return df

In [14]:
aapl_df = macd(aapl_df, 12, 26)
aapl_df

,Date,Close,Open,High,Low,SMA_20,EMA_20,BB_upper_20,BB_lower_20,MACD_12_26,MACDsign_12_26,MACDdiff_12_26
0,"Dec 29, 2020",136.53,138.05,138.78,135.53,126.7815,127.573094,136.337708,117.225292,4.107426,3.134253,0.973173
1,"Dec 28, 2020",136.69,133.99,137.34,133.52,125.9075,126.630262,134.889829,116.925171,3.794332,2.890959,0.903372
2,"Dec 24, 2020",131.97,131.32,133.46,131.10,124.9025,125.571342,133.282848,116.522152,3.323228,2.665116,0.658112
3,"Dec 23, 2020",130.96,132.18,132.32,130.83,124.1055,124.897799,132.685294,115.525706,3.154172,2.500588,0.653584
4,"Dec 22, 2020",131.88,131.68,134.40,129.66,123.3160,124.259672,132.142442,114.489558,2.995353,2.337192,0.658161
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,"Feb 22, 2001",0.34,0.34,0.35,0.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4996,"Feb 21, 2001",0.34,0.33,0.36,0.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4997,"Feb 20, 2001",0.33,0.34,0.35,0.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4998,"Feb 16, 2001",0.34,0.34,0.35,0.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Relative Strength Indicators

### RSI
Relative Strength Index  
https://www.fmlabs.com/reference/default.htm?url=RSI.htm  
https://github.com/Crypto-toolbox/pandas-technical-indicators/blob/master/technical_indicators.py#L313

In [20]:
def RSI(df, time_period=20):
    """Calculate Relative Strength Index(RSI) for given data.
    :param df: pandas.DataFrame
    :param n: 
    :return: pandas.DataFrame
    """
    i = 0
    UpI = [0]
    DoI = [0]
    while i + 1 <= df.index[-1]:
        UpMove = df.loc[i + 1, 'High'] - df.loc[i, 'High']
        DoMove = df.loc[i, 'Low'] - df.loc[i + 1, 'Low']
        if UpMove > DoMove and UpMove > 0:
            UpD = UpMove
        else:
            UpD = 0
        UpI.append(UpD)
        if DoMove > UpMove and DoMove > 0:
            DoD = DoMove
        else:
            DoD = 0
        DoI.append(DoD)
        i = i + 1
    UpI = pd.Series(UpI).iloc[::-1]
    DoI = pd.Series(DoI).iloc[::-1]
    PosDI = UpI.ewm(span=time_period, min_periods=time_period).mean().iloc[::-1]
    NegDI = DoI.ewm(span=time_period, min_periods=time_period).mean().iloc[::-1]
    RSI = PosDI / (PosDI + NegDI)
    df['RSI_' + str(time_period)] = RSI
    return df

In [21]:
aapl_df = RSI(aapl_df, 20)
aapl_df

,Date,Close,Open,High,Low,SMA_20,EMA_20,BB_upper_20,BB_lower_20,MACD_12_26,MACDsign_12_26,MACDdiff_12_26,RSI_20
0,"Dec 29, 2020",136.53,138.05,138.78,135.53,126.7815,127.573094,136.337708,117.225292,4.107426,3.134253,0.973173,0.232970
1,"Dec 28, 2020",136.69,133.99,137.34,133.52,125.9075,126.630262,134.889829,116.925171,3.794332,2.890959,0.903372,0.232970
2,"Dec 24, 2020",131.97,131.32,133.46,131.10,124.9025,125.571342,133.282848,116.522152,3.323228,2.665116,0.658112,0.261353
3,"Dec 23, 2020",130.96,132.18,132.32,130.83,124.1055,124.897799,132.685294,115.525706,3.154172,2.500588,0.653584,0.301346
4,"Dec 22, 2020",131.88,131.68,134.40,129.66,123.3160,124.259672,132.142442,114.489558,2.995353,2.337192,0.658161,0.306074
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,"Feb 22, 2001",0.34,0.34,0.35,0.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4996,"Feb 21, 2001",0.34,0.33,0.36,0.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4997,"Feb 20, 2001",0.33,0.34,0.35,0.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4998,"Feb 16, 2001",0.34,0.34,0.35,0.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Info

In [22]:
aapl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date            5000 non-null   object 
 1   Close           5000 non-null   float64
 2   Open            5000 non-null   float64
 3   High            5000 non-null   float64
 4   Low             5000 non-null   float64
 5   SMA_20          4981 non-null   float64
 6   EMA_20          4981 non-null   float64
 7   BB_upper_20     4981 non-null   float64
 8   BB_lower_20     4981 non-null   float64
 9   MACD_12_26      4975 non-null   float64
 10  MACDsign_12_26  4967 non-null   float64
 11  MACDdiff_12_26  4967 non-null   float64
 12  RSI_20          4981 non-null   float64
dtypes: float64(12), object(1)
memory usage: 507.9+ KB


In [23]:
aapl_df.to_csv('aapl_stock_price_with_indicators_with_nan.csv', index=False)

### Drop nan rows

In [25]:
aapl_df = aapl_df.dropna()
aapl_df

,Date,Close,Open,High,Low,SMA_20,EMA_20,BB_upper_20,BB_lower_20,MACD_12_26,MACDsign_12_26,MACDdiff_12_26,RSI_20
0,"Dec 29, 2020",136.53,138.05,138.78,135.53,126.7815,127.573094,136.337708,117.225292,4.107426,3.134253,0.973173,0.232970
1,"Dec 28, 2020",136.69,133.99,137.34,133.52,125.9075,126.630262,134.889829,116.925171,3.794332,2.890959,0.903372,0.232970
2,"Dec 24, 2020",131.97,131.32,133.46,131.10,124.9025,125.571342,133.282848,116.522152,3.323228,2.665116,0.658112,0.261353
3,"Dec 23, 2020",130.96,132.18,132.32,130.83,124.1055,124.897799,132.685294,115.525706,3.154172,2.500588,0.653584,0.301346
4,"Dec 22, 2020",131.88,131.68,134.40,129.66,123.3160,124.259672,132.142442,114.489558,2.995353,2.337192,0.658161,0.306074
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4962,"Apr 10, 2001",0.39,0.37,0.41,0.37,0.3765,0.374418,0.417680,0.335320,0.004108,0.006003,-0.001894,0.498603
4963,"Apr 09, 2001",0.37,0.37,0.38,0.36,0.3745,0.372778,0.416790,0.332210,0.003071,0.006476,-0.003405,0.543482
4964,"Apr 06, 2001",0.37,0.37,0.38,0.36,0.3725,0.373070,0.419236,0.325764,0.003739,0.007328,-0.003588,0.591666
4965,"Apr 05, 2001",0.37,0.37,0.40,0.36,0.3720,0.373394,0.419061,0.324939,0.004538,0.008225,-0.003687,0.591666


In [26]:
aapl_df.to_csv('aapl_stock_price_with_indicators.csv', index=False)

# Visualization